# Defining Physics Lists

We need to provide a type to the `G4JLApplication` that represents the selected physics list. The easiest is to use one of the pre-packaged physics list (e.g. `FTFP_BERT`, `QGS_BIC`, `QBBC` and others can be added on demand). The toolkit will use the provided type to instantiate the physics list at the right moment during the initialization.

In [ ]:
using Geant4
using Test

## Using Standard Physics Lists
Lets play a bit one of these pre-packaged physics list

In [ ]:
fl = FTFP_BERT()
@test fl isa G4VUserPhysicsList
DumpList(fl)

It is normal that created instance by hand is empty. It needs to be created by the `G4RunManager` during initialization. Let's try to do it.

In [ ]:
#---The next will be hidden in new versions-------------------------
# create a empty world filled with vacumm
struct World <: G4JLDetector end
using Geant4.PhysicalConstants: universe_mean_density
using Geant4.SystemOfUnits: g, mole, kelvin, pascal
function bigbang(::World)
    vacuum = G4Material("Vacuum", z=1., a=1.01g/mole, density=universe_mean_density, state=kStateGas, 
                        temperature=2.73*kelvin, pressure=3.e-18*pascal)
    G4PVPlacement(nothing, G4ThreeVector(),
                  G4LogicalVolume(G4Box("world", 1000,1000,1000), move!(vacuum), "World"),
                  "World", nothing, false, 0, false)
end  
Geant4.getConstructor(::World) = bigbang
#---end-------------------------------------------------------------

In [ ]:

app = G4JLApplication(
    detector = World(),               # should be the default
    generator = G4JLGunGenerator(),   # should be the default
    physics_type = FTFP_BERT
)

In [ ]:
configure(app)
initialize(app)

In [ ]:
DumpList(app.physics)

## Defining a custom physics list
The way we can achieve it is by defining a dummy `struct` inheriting from `G4VUserPhysicsList` that provides a default constructor. In the constructor we can build the physics list and return an object that inherits from `G4VUserPhysicsList` to comply with the interface.

Here is an example:

In [ ]:
ui`/run/verbose 1`
app = nothing
GC.gc()
G4RunManager!GetRunManager()

In [ ]:
struct MyPhysicsList <: G4VUserPhysicsList
    function MyPhysicsList(verbose)
        pl = G4VModularPhysicsList()
        RegisterPhysics(pl, move!(G4EmStandardPhysics(verbose)))
        RegisterPhysics(pl, move!(G4OpticalPhysics(verbose)))
        return pl
    end 
end

In [ ]:
app = G4JLApplication(
    detector = World(),               # should be the default
    generator = G4JLGunGenerator(),   # should be the default
    physics_type = MyPhysicsList,
)
configure(app)
initialize(app)

In [ ]:
DumpList(app.physics)